In [3]:
# Look at daysim outputs to relate value of time to income generally
import pandas as pd
import h5py

In [39]:
daysim = h5py.File(r'R:\SoundCast\releases\TransportationFutures2010\outputs\daysim_outputs.h5')


In [40]:
def build_df(h5file, h5table, var_dict, survey_file=False):
    ''' Convert H5 into dataframe '''
    data = {}
    if survey_file:
        # survey h5 have nested data structure, different than daysim_outputs
        for col_name, var in var_dict.iteritems():
            data[col_name] = [i[0] for i in h5file[h5table][var][:]]
    else:
        for col_name, var in var_dict.iteritems():
            data[col_name] = [i for i in h5file[h5table][var][:]]

    return pd.DataFrame(data)

In [52]:
tripdict={'Household ID': 'hhno',
            'Person Number': 'pno',
            'Travel Time':'travtime',
            'Travel Cost': 'travcost',
            'Travel Distance': 'travdist',
            'Mode': 'mode',
            'Purpose':'dpurp',
            'Departure Time': 'deptm',
            'Expansion Factor': 'trexpfac'}

In [53]:
daysim['Trip']['hhno'][:][0]

14100001

In [54]:
trip = build_df(h5file=daysim, h5table='Trip', var_dict=tripdict, survey_file=True)

In [55]:
hhdict={'Household ID': 'hhno',
        'Household Size': 'hhsize',
        'Household Vehicles': 'hhvehs',
        'Household Workers': 'hhwkrs',
        'Household Income': 'hhincome',
        'Household TAZ': 'hhtaz',
        'Expansion Factor': 'hhexpfac'}

In [56]:
hh = build_df(h5file=daysim, h5table='Household', var_dict=hhdict, survey_file=False)

In [57]:
# Join household to trip data to get income
trip_hh = pd.merge(trip,hh, on='Household ID')

In [58]:
df = trip_hh

In [59]:
df['VOT Bin'] = pd.cut(df['VOT'], bins=[0,15,25,99999], right=True, labels=[1,2,3], retbins=False, precision=3, include_lowest=True)

KeyError: 'VOT'

In [29]:
df.groupby('VOT Bin').mean()['Household Income']

VOT Bin
1           84554.525595
2          107971.862880
3          135164.979754
Name: Household Income, dtype: float64